In [1]:
import subprocess
import sys

In [2]:
def install_package(package):
    """Installs the package using pip"""
    subprocess.check_call([sys.executable, '-m', 'pip','install',package])

In [3]:
def check_and_install_dependencies(packages):
    """Checks if packages are installed, installs them if not"""
    for package in packages:
        try:
            __import__(package.split('==')[0])
            print(f"{package} is alreday installed")
        except ImportError:
            print(f'{package} not found. Installing...')
            install_package(package)
            print(f'{package} has been installed.')

In [4]:
import ctypes
import serial
import time
import serial.tools.list_ports
from collections import Counter

In [5]:
check_and_install_dependencies(['pyserial==3.5','serial==0.0.97','psycopg2-binary'])

pyserial==3.5 not found. Installing...



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip


pyserial==3.5 has been installed.
serial==0.0.97 is alreday installed
psycopg2-binary not found. Installing...
psycopg2-binary has been installed.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [6]:
data_original=open('data.csv','r').readlines()

In [7]:
# Function to extract selected columns
def extract_columns(data, selected_columns,delimiter=';'):
    '''
    Extracts the intended data given the indeces in selected_columns
    Determines which is the current state type.
    '''
    columns = data.split(delimiter)  
    result = [str(int(columns[i])/1000) if i==6 else columns[i] for i in selected_columns]
    if result[2]=='1':
        estado='charging'
    elif result [2]=='2':
        estado='discharging'
    elif result [2]=='4':
        estado='rest'
    elif result[2]=='6':
        estado='finished'
    result[2]=estado
    return delimiter.join(result),estado #return the data as string

def save_file(estado,bateria,capacidad,ciclo,data):
    '''
    Function for saving the data file (.csv)
    By data analisis, each current state is compared to the previous four (4) states. 
    When the 5 states changes, then it's certain the battery power state has changed
    '''
    estados_pasados.append(estado)
    #determine the correct battery state (charging, resting, discharging) to save data to
    if len(estados_pasados)>4:
        if all(x==estado for x in estados_pasados):
            file_name=f"{bateria}{estado}_{capacidad}_{ciclo}.csv"
        else:
            most_common_elem, count = Counter(estados_pasados).most_common(1)[0]
            if most_common_elem=="charging" and count==3 and estado=='charging':
                ciclo+=1
            file_name=f"{bateria}{most_common_elem}_{capacidad}_{ciclo}.csv"
        if not(file_name in dict_data):
            dict_data[file_name]=[]
            state_file = open(file_name, "w")
            state_file.write('date;time;battery_state;voltage[V];current[mA];capacity[mAh]'+'\n')#setting column titles
            state_file.flush()
        dict_data[file_name].append(data)
        estados_pasados.pop(0)
        
        #writing data to the specific file
        try:
            state_file = open(file_name, "a")
        except:
            print("book is already open")
        finally:
            state_file.write(data+'\n')
            state_file.flush()
    return ciclo

In [8]:
log_to_file=True
estados_pasados=[]
dict_data={}
columns_to_extract=[0,1,3,6,7,16]#date, time, battery_state(charing/resting/discharging),voltage(V),current(mA),capacity(mAh)
bateria='1'
capacidad='1000'
ciclo=1

if log_to_file:
    log_file = open(f"data_original_{bateria}_{capacidad}_{ciclo}.csv", "w")
    log_file.write('date;time;battery_state;voltage[V];current[mA];capacity[mAh]'+'\n')#setting column titles
    log_file.flush()
else:
    log_file = None
                
for a in data_original[:-1]:
    data,estado=extract_columns(a,columns_to_extract)
    if estados_pasados==['finished','finished','finished','finished']:
        print("finished cicles \nclosing program")
        print(a.split(';')[0])
        break
        
    if log_to_file:
        #original data
        if log_file:
            log_file.write(data + '\n')
            log_file.flush()
            ciclo=save_file(estado,bateria,capacidad,ciclo,data)   

finished cicles 
closing program
2025-09-23


In [9]:
#read from CSV to DB

In [13]:
import os
import csv
from pathlib import Path
from datetime import datetime
txt_files = [f for f in os.listdir(os.getcwd()) if f.endswith('.csv') and os.path.isfile(os.path.join(os.getcwd(), f))]

In [17]:
#reading data and preparing for the DB
def infer_data_type(value: str) -> str:
    """Guess the data type from a single value."""
    value = value.strip()

    if value == "":
        return "TEXT"  # fallback for empty strings

    if value.isdigit():
        return "INTEGER"
    
    try:
        float(value)
        return "FLOAT"
    except ValueError:
        pass

    for fmt in ("%Y-%m-%d", "%Y-%m-%d %H:%M:%S"):
        try:
            datetime.strptime(value, fmt)
            return "TIMESTAMP"
        except ValueError:
            continue

    return "TEXT"

def normalize_column_name(name: str) -> str:
    """Make column names safe for SQL."""
    return name.strip().lower().replace(" ", "_")

def get_column_types_from_csv(csv_file_path: str) -> dict:
    """Reads CSV and returns a dict of {column_name: inferred_type}."""
    csv_path = Path(csv_file_path)
    if not csv_path.exists():
        raise FileNotFoundError(f"{csv_file_path} not found.")

    with csv_path.open('r', newline='', encoding='utf-8') as f:
        reader = csv.DictReader(f,delimiter=';')
        first_row = next(reader)

    col_types = {}
    for col, val in first_row.items():
        col_name = normalize_column_name(col)
        col_type = infer_data_type(val)
        col_types[col_name] = col_type

    return col_types

In [18]:
for a in txt_files:
    print(get_column_types_from_csv(a))

{'2025-09-23': 'TIMESTAMP', '08:51:02': 'TEXT', '$1': 'TEXT', '1': 'INTEGER', '': 'TEXT', '12000': 'INTEGER', '4190': 'INTEGER', '7': 'INTEGER', '0': 'INTEGER', '328': 'INTEGER', '6': 'INTEGER', '40': 'INTEGER'}
{'date': 'TIMESTAMP', 'time': 'TEXT', 'battery_state': 'TEXT', 'voltage[v]': 'FLOAT', 'current[ma]': 'INTEGER', 'capacity[mah]': 'INTEGER'}
{'date': 'TIMESTAMP', 'time': 'TEXT', 'battery_state': 'TEXT', 'voltage[v]': 'FLOAT', 'current[ma]': 'INTEGER', 'capacity[mah]': 'INTEGER'}
{'date': 'TIMESTAMP', 'time': 'TEXT', 'battery_state': 'TEXT', 'voltage[v]': 'FLOAT', 'current[ma]': 'INTEGER', 'capacity[mah]': 'INTEGER'}


AttributeError: 'NoneType' object has no attribute 'strip'